In [1]:
import requests
import re
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
import time
import os
import json

In [2]:
user_agent = {'User-agent': 'Mozilla/5.0'}

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)

In [3]:
def get_film_data(film_id):
    
    delimiter = '+_+'
    
    film_url = "http://www.imdb.com/title/" + film_id
    driver.get(film_url)
    
    response = requests.get(film_url, headers = user_agent)
    html = response.text
    soup = bs(html, 'lxml')
    
    film_dict = {}
    
    try:
        genre_selector = "//*[@id='titleStoryLine']//a[contains(@href, '/genre/')]"
        film_dict['genres'] = ''
        for genre in driver.find_elements_by_xpath(genre_selector):
            film_dict['genres'] = film_dict['genres'] + delimiter + genre.text
    except NoSuchElementException:
        film_dict['genres'] = None
        
    try:
        title_selector = "//*[@id='title-overview-widget']//*[@itemprop='name']"
        film_dict['title'] = driver.find_element_by_xpath(title_selector).text
    except NoSuchElementException:
        film_dict['title'] = None
    
    try:
        year_selector = '//*[@id="titleYear"]'
        film_dict['year'] = driver.find_element_by_xpath(year_selector).text.split('(')[1].split(')')[0]
    except NoSuchElementException:
        film_dict['year'] = None  
    
    try:
        rating_selector = "//*[@id='title-overview-widget']//*[@itemprop='ratingValue']"
        film_dict['rating'] = driver.find_element_by_xpath(rating_selector).text
    except NoSuchElementException:
        film_dict['rating'] = None  
   
    try:
        votes_selector = "//*[@id='title-overview-widget']//*[@itemprop='ratingCount']"
        film_dict['votes'] = driver.find_element_by_xpath(votes_selector).text
    except NoSuchElementException:
        film_dict['votes'] = None     
    
    try:
        directors_selector = "//*[@id='title-overview-widget']//*[@itemprop='director']"
        film_dict['directors'] = ''
        for director in driver.find_elements_by_xpath(directors_selector):
            film_dict['directors'] = film_dict['directors'] + delimiter + director.text.split('(')[0].strip()
    except NoSuchElementException:
        film_dict['directors'] = None    
  
    try:
        writers_selector = "//*[@id='title-overview-widget']//*[@itemprop='creator']"
        film_dict['writers'] = ''
        for writer in driver.find_elements_by_xpath(writers_selector):
            film_dict['writers'] = film_dict['writers'] + delimiter + writer.text.split('(')[0].strip()
    except NoSuchElementException:
        film_dict['writers'] = None     

    try:
        actors_selector = "//*[@id='titleCast']//*[@itemprop='name']"
        film_dict['actors'] = ''
        for actor in driver.find_elements_by_xpath(actors_selector):
            film_dict['actors'] = film_dict['actors'] + delimiter + actor.text
    except NoSuchElementException:
        film_dict['actors'] = None          

    try:
        keywords_selector = "//*[@class='see-more inline canwrap']//*[@itemprop='keywords']"
        film_dict['keywords'] = ''
        for keyword in driver.find_elements_by_xpath(keywords_selector):
            film_dict['keywords'] = film_dict['keywords'] + delimiter + keyword.text
    except NoSuchElementException:
        film_dict['keywords'] = None       

    try:
        mpaa_selector = "//meta[@itemprop='contentRating']"
        film_dict['mpaa'] = driver.find_element_by_xpath(mpaa_selector).get_attribute('content')
    except NoSuchElementException:
        film_dict['mpaa'] = None      

    try:
        release_date_selector = "//meta[@itemprop='datePublished']"
        film_dict['release_date'] = driver.find_element_by_xpath(release_date_selector).get_attribute('content')
    except NoSuchElementException:
        film_dict['release_date'] = None     

    try:
        languages_selector = "//*[@id='titleDetails']//a[contains(@href, 'language')]"
        film_dict['languages'] = ''
        for language in driver.find_elements_by_xpath(languages_selector):
            film_dict['languages'] = film_dict['languages'] + delimiter + language.text
    except NoSuchElementException:
        film_dict['languages'] = None   
    
    try:
        countries_selector = "//*[@id='titleDetails']//a[contains(@href, 'country_of_origin')]"
        film_dict['countries'] = ''
        for country in driver.find_elements_by_xpath(countries_selector):
            film_dict['countries'] = film_dict['countries'] + delimiter + country.text
    except NoSuchElementException:
        film_dict['countries'] = None 
    
    try:
        runtime_selector = "//*[@id='titleDetails']//*[@itemprop='duration']"
        film_dict['runtime'] = int(driver.find_element_by_xpath(runtime_selector).text.split(' ')[0])
    except NoSuchElementException:
        film_dict['runtime'] = None 
     
    film_dict['budget'] = None
    film_dict['gross_usa'] = None
    for h4 in soup.find_all('h4'):
        if "Budget:" in h4:
            film_dict['budget'] = h4.next_sibling        
        if "Gross USA:" in h4:
            film_dict['gross_usa'] = h4.next_sibling
            
    return film_dict

In [4]:
with open("film_ids.json","r") as f:
      data = f.read()
        
film_ids = json.loads(data)

In [5]:
dict_list = []
dummy = 0

for film_id in film_ids[3000:4000]:
    print(dummy)
    dict_list.append(get_film_data(film_id))
    dummy = dummy + 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [6]:
import pandas as pd

data = json.dumps(dict_list)

with open("film_data_4.json","w") as f:
      f.write(data)

with open("film_data_4.json","r") as f:
      data = f.read()
        
d = json.loads(data)

df = pd.DataFrame(d)

df

,actors,budget,countries,directors,genres,gross_usa,keywords,languages,mpaa,rating,release_date,runtime,title,votes,writers,year
0,+_+Alicia Silverstone+_+Stacey Dash+_+Brittany...,"$12,000,000\n",+_+USA,+_+Amy Heckerling,+_+Comedy+_+Romance,"$56,631,572\n",+_+high school+_+modern day adaptation+_+refer...,+_+English+_+Spanish,PG-13,6.8,1995-07-19,97.0,Clueless (1995),"140,486",+_+Amy Heckerling,1995
1,+_+Jason James Richter+_+Francis Capra+_+Mary ...,None,+_+France+_+USA,+_+Dwight H. Little,+_+Adventure+_+Drama+_+Family,"$30,077,111\n",+_+warner bros+_+whale+_+love+_+half brother+_...,+_+English+_+Malay+_+Indonesian,PG,4.9,1995-07-19,95.0,Free Willy 2: The Adventure Home (1995),"14,214",+_+Keith Walker+_+Karen Janszen,1995
2,+_+Julia Roberts+_+Dennis Quaid+_+Robert Duval...,None,+_+USA,+_+Lasse Hallström,+_+Comedy+_+Drama+_+Romance,"$50,892,160\n",+_+female protagonist+_+wealth+_+unfaithfulnes...,+_+English,R,5.7,1995-08-04,106.0,Something to Talk About (1995),"11,245",+_+Callie Khouri,1995
3,+_+Jesse Cameron-Glickenhaus+_+Pat Morita+_+Jo...,None,+_+USA,+_+James Glickenhaus,+_+Adventure+_+Sci-Fi+_+Western,None,+_+outer space+_+street shootout+_+arizona+_+o...,+_+English,PG-13,3.8,1995-12-27,100.0,Timemaster (1995),329,+_+James Glickenhaus,1995
4,+_+Billy Zane+_+Mia Sara+_+James Russo+_+James...,None,+_+USA,+_+Strathford Hamilton,+_+Action+_+Drama+_+Thriller,None,+_+bare breasts+_+nudity+_+caucasian+_+sex sce...,+_+English,R,4.7,1995-07-23,93.0,The Set Up (1995),408,+_+James Hadley Chase+_+Michael Thoma,1995
5,+_+Sandra Bullock+_+Jeremy Northam+_+Dennis Mi...,"$22,000,000\n",+_+USA,+_+Irwin Winkler,+_+Action+_+Crime+_+Drama+_+Mystery+_+Thriller,"$50,727,965\n",+_+computer+_+identity+_+identity theft+_+comp...,+_+English+_+Spanish,PG-13,5.9,1995-07-28,114.0,The Net (1995),"52,642","+_+John Brancato,+_+Michael Ferris",1995
6,+_+Robert Patrick+_+Joanna Pacula+_+Vyto Rugin...,None,+_+USA+_+Romania,+_+Scott McGinnis,+_+Horror+_+Thriller,None,+_+female nudity+_+hunt+_+curse+_+private inve...,+_+English,R,4.8,1995-07-25,93.0,Last Gasp (1995),371,+_+Pierce Milestone+_+Pierce Milestone,1995
7,+_+Michael Biehn+_+Gabriel Arcand+_+Alexandra ...,None,+_+USA,+_+Gilles Carle,+_+Action,None,,+_+English,PG-13,4.8,1995-07-25,NaN,Blood of the Hunter (1995),6,"+_+Gilles Carle,+_+James Oliver Curwood",1995
8,+_+Andrew Smith+_+John Bluthal+_+Ray Bright+_+...,None,+_+USA,+_+Ted Nicolaou,+_+Family+_+Fantasy+_+Horror,None,+_+leprechaun+_+reference to rembrandt+_+refer...,+_+English,PG,4.3,1995-07-26,NaN,Leapin' Leprechauns! (1995),200,+_+Charles Band+_+Michael McGann,1995
9,+_+Kevin Costner+_+Chaim Jeraffi+_+Rick Aviles...,"$175,000,000\n",+_+USA,+_+Kevin Reynolds,+_+Action+_+Adventure+_+Sci-Fi+_+Thriller,"$88,246,220\n",+_+sea+_+smoker+_+future+_+water+_+sail,+_+English,PG-13,6.1,1995-07-28,135.0,Waterworld (1995),"156,337","+_+Peter Rader,+_+David Twohy",1995
